In [65]:
import cv2
import numpy as np
from keras.models import model_from_json
from transformers import pipeline
import pickle
import sounddevice as sd
from scipy.io.wavfile import write
import librosa
import numpy as np
import os
import pandas as pd
import speech_recognition as sr
import threading
from threading import Thread
from multiprocessing.pool import ThreadPool
import concurrent.futures
import wavio
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification



aud_emot = 'Undefined'
text_emot = 'Undefined'
vid_emot = 'Undefined'

tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")


#######################################TEXT###########################################






#classifier = pipeline('sentiment-analysis')  #distilledbert-base-uncased-finetuned-sst-2-english

   #for different model use cases from model hub of huggingface
#print(classifier_text("We love the software developed by you "))



def txt():
    r = sr.Recognizer()

    hellow=sr.AudioFile('output.wav')
    with hellow as source:
        audio = r.record(source)
    s = r.recognize_google(audio)
    
    inputs = tokenizer(s, return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits

    predicted_class_id = logits.argmax().item()
    global text_emot
    
    text_emot = model.config.id2label[predicted_class_id]
    


#######################################AUDIO###########################################







filename = 'mlp_aud.sav'
loaded_model = pickle.load(open(filename, 'rb'))

def extract_feature(file_name):
    
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    result = np.array([])
    
    stft = np.abs(librosa.stft(X))
    chromas = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chromas))

    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    result=np.hstack((result, mfccs))

    mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate, n_mels=128).T, axis=0)
    result = np.hstack((result, mels))
    
        
    return result



fs = 44100  # Sample rate
seconds = 3  # Duration of recording




def aud():
    
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
    sd.wait()  # Wait until recording is finished
    r = sr.Recognizer()
    #aud_to_text = r.record('output.wav')
    #result = 'Unknown'
    #print(type(myrecording))
    #with myrecording as source: 
     #   r.adjust_for_ambient_noise(source) 
      #  audio = r.record(source)
      #  result = r.recognize_google(audio)
    #text_emot_loc = classifier_text(result)
    #write('output.wav', fs, myrecording)  # Save as WAV file 
       #r = sr.Recognizer()
    
    wavio.write('output.wav', myrecording, fs ,sampwidth=2)
    
        
        
    test = extract_feature('output.wav')
    test_pd = pd.DataFrame(test)
    test = extract_feature('output.wav')
    ##test.reshape(-1,1)
    test_pd = pd.DataFrame(test)
    test_t = test_pd.T
    aud_emot_loc = loaded_model.predict(test_t)
    
    global aud_emot
    aud_emot = aud_emot_loc[0]
    txt()


#######################################VIDEO###########################################





emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file = open('emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)


emotion_model.load_weights("emotion_model.h5")
#print("Loaded model from disk")


def vid():

    cap = cv2.VideoCapture(0)
        # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
   
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    vid_emot_loc = 'Unknown'
# take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

            # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        vid_emot1 = int(np.argmax(emotion_prediction))
        vid_emot_loc = emotion_dict[vid_emot1]
        #cv2.putText(frame, emotion_dict[vid_emot], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        #cv2.imshow('Emotion Detection', frame)
        #if cv2.waitKey(1) & 0xFF == ord('q'):
        #   break

    cap.release()
    cv2.destroyAllWindows()
    
    global vid_emot
    vid_emot = vid_emot_loc
    






###main###

thread_1 = Thread(target = vid)
thread_2 = Thread(target = aud)
  
thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()


print(aud_emot)
print(text_emot)
print(vid_emot)


1/1 [==============================] - 0s 94ms/step
result2:
{   'alternative': [{'confidence': 0.88687539, 'transcript': 'hello system'}],
    'final': True}
disgust
optimism
Surprised
